In [ ]:
# We construct originality & Generality

In [2]:
import base64
import datetime
import pandas as pd
import json
from google.cloud import bigquery

client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: research-311404


In [13]:
sql = """
CREATE TABLE
  `research-311404.Generality_Consolidated.patent_reference` AS
SELECT distinct inventor_id,t1.patent_id,t2.citation_id as reference, year,t3.subsection_id as patent_category, t3.group_id as patent_group, t4.subsection_id as reference_category,t4.group_id as reference_group
from (SELECT patent_id,year,inventor_id FROM `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`) as t1
join  `patents-public-data.patentsview.uspatentcitation` as t2
on t1.patent_id=t2.patent_id
join  `research-311404.PatTV_Construction.first_cpc_category` as t3
on t1.patent_id=t3.patent_id
join  `research-311404.PatTV_Construction.first_cpc_category` as t4
on t2.citation_id=t4.patent_id
"""

job = client.query(sql)  # API request.
job.result()
table_id = "research-311404.Generality_Consolidated.patent_reference"
table = client.get_table(table_id)
print("Table {} now contains {} rows".format(table_id, table.num_rows))

Table research-311404.Generality_Consolidated.patent_reference now contains 282034968 rows


In [ ]:
sql = """
CREATE TABLE
  `research-311404.Generality_Consolidated.originality` AS
SELECT t1.patent_id, (1 - SUM(POWER(ncited_group/ncited_pat,2))) AS originality
FROM (SELECT patent_id,reference_group,
count(*) as ncited_group FROM `research-311404.Generality_Consolidated.patent_reference` 
group by patent_id, reference_group)t1 
LEFT JOIN
(SELECT patent_id,count(*) as ncited_pat FROM `research-311404.Generality_Consolidated.patent_reference` 
group by patent_id)t2
ON t1.patent_id = t2.patent_id
GROUP BY t1.patent_id
"""
job = client.query(sql)  # API request.
job.result()
table_id = "research-311404.Generality_Consolidated.originality"
table = client.get_table(table_id)
print("Table {} now contains {} rows".format(table_id, table.num_rows))

In [16]:
sql = """
CREATE TABLE
  `research-311404.Generality_Consolidated.patent_citation` AS
SELECT distinct inventor_id,t1.patent_id,t2.patent_id as citation,year,t3.subsection_id as patent_category, t3.group_id as patent_group,
t4.subsection_id as citation_category,t4.group_id as citation_group
from (SELECT patent_id,year,inventor_id FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`) as t1
join  `patents-public-data.patentsview.uspatentcitation` as t2
on t1.patent_id=t2.citation_id
join  `research-311404.PatTV_Construction.first_cpc_category` as t3
on t1.patent_id=t3.patent_id
join  `research-311404.PatTV_Construction.first_cpc_category` as t4
on t2.patent_id=t4.patent_id
"""
job = client.query(sql)  # API request.
job.result()
table_id = "research-311404.Generality_Consolidated.patent_citation"
table = client.get_table(table_id)
print("Table {} now contains {} rows".format(table_id, table.num_rows))

Table research-311404.Generality_Consolidated.patent_citation now contains 235487155 rows


In [17]:
sql = """
CREATE TABLE
  `research-311404.Generality_Consolidated.generality` AS
SELECT t1.patent_id, (1 - SUM(POWER(nciting_group/nciting_pat,2))) AS generality
FROM (SELECT patent_id,citation_group,
count(*) as nciting_group FROM `research-311404.Generality_Consolidated.patent_citation` 
group by patent_id, citation_group)t1 
LEFT JOIN
(SELECT patent_id,count(*) as nciting_pat FROM `research-311404.Generality_Consolidated.patent_citation` 
group by patent_id)t2
ON t1.patent_id = t2.patent_id
GROUP BY t1.patent_id
"""
job = client.query(sql)  # API request.
job.result()
table_id = "research-311404.Generality_Consolidated.generality"
table = client.get_table(table_id)
print("Table {} now contains {} rows".format(table_id, table.num_rows))

Table research-311404.Generality_Consolidated.generality now contains 5123492 rows
